<a href="https://colab.research.google.com/github/fcoliveira-utfpr/climate_parana/blob/main/analises_clima_parana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Início - Bibliotecas
---

In [26]:
#importantando bibliotecas
import numpy as np
import pandas as pd

# Abrindo dados
---

In [36]:
url = "https://raw.githubusercontent.com/fcoliveira-utfpr/climate_parana/refs/heads/main/dados_concatenados.csv"
df = pd.read_csv(url, sep = ",")
df

,estado,lat,lon,municipio,valor,variavel
0,Parana,-23.311363,-50.306176,Abatia,210.718086,def
1,Parana,-24.781986,-48.819381,Adrianopolis,54.045608,def
2,Parana,-26.025501,-49.309155,Agudos Do Sul,33.173108,def
3,Parana,-25.284879,-49.322794,Almirante Tamandare,35.327444,def
4,Parana,-24.812720,-52.691907,Altamira Do Parana,99.582733,def
...,...,...,...,...,...,...
2389,Parana,-23.538200,-53.825424,Vila Alta,28.208377,tmmx
2390,Parana,-25.408510,-52.236439,Virmond,24.368879,tmmx
2391,Parana,-26.277366,-52.804057,Vitorino,24.684439,tmmx
2392,Parana,-23.853953,-49.783944,Wenceslau Braz,25.967448,tmmx
